In [87]:
#In this assignment, students will be using the K-nearest neighbors algorithm to predict 
#how many points NBA players scored in the 2013-2014 season.
import pandas
with open("nba_2013.csv", 'r') as csvfile:
    nba = pandas.read_csv(csvfile)

# The names of all the columns in the data.
print(nba.columns.values)
#nba.drop('season', axis=1)
#nba.drop('season_end', axis=1)

['player' 'age' 'g' 'gs' 'mp' 'fg' 'fga' 'fg.' 'x3p' 'x3pa' 'x3p.' 'x2p'
 'x2pa' 'x2p.' 'efg.' 'ft' 'fta' 'ft.' 'orb' 'drb' 'trb' 'ast' 'stl' 'blk'
 'tov' 'pf' 'pts' 'season_end']


In [88]:
# Select Lebron James from our dataset
import math
selected_player = nba[nba["player"] == "LeBron James"].iloc[0]

# Choose only the numeric columns (we'll use these to compute euclidean distance)
distance_columns = ['age', 'g', 'gs', 'mp', 'fg', 'fga', 'fg.', 'x3p', 'x3pa', 'x3p.', 'x2p', 'x2pa', 'x2p.', 'efg.', 'ft', 'fta', 'ft.', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts']

def euclidean_distance(row):
    """
    A simple euclidean distance function
    """
    inner_value = 0
    for k in distance_columns:
        inner_value += (row[k] - selected_player[k]) ** 2
    return math.sqrt(inner_value)

# Find the distance from each player in the dataset to lebron.
lebron_distance = nba.apply(euclidean_distance, axis=1)
lebron_distance

0       787.613501
1       485.856006
2         0.000000
3       679.710997
4       568.833040
5       709.937352
6       674.207716
7       543.179606
8       607.684190
9       645.576517
10      782.023066
11     1005.517368
12      960.443178
13      986.609376
14      716.243023
15      711.192700
16      891.808844
17     1092.834411
18      959.465515
19     1055.479535
20      913.811828
21     1086.244015
22      949.668916
23     1162.762249
24     1027.760224
25     1023.849173
26     1283.191742
27     1189.554979
28     1096.809483
29     1150.505137
          ...     
451            NaN
452    4224.419271
453            NaN
454    4143.814549
455            NaN
456            NaN
457            NaN
458    4221.985029
459            NaN
460            NaN
461            NaN
462    4250.785747
463            NaN
464            NaN
465            NaN
466            NaN
467            NaN
468            NaN
469            NaN
470            NaN
471    4260.263523
472    4257.

In [89]:
    
# Select only the numeric columns from the NBA dataset
nba_numeric = nba[distance_columns]

# Normalize all of the numeric columns
nba_normalized = (nba_numeric - nba_numeric.mean()) / nba_numeric.std()

In [90]:
from scipy.spatial import distance
import pandas as pd

# Fill in NA values in nba_normalized
nba_normalized.fillna(0, inplace=True)

# Find the normalized vector for lebron james.
lebron_normalized = nba_normalized[nba["player"] == "LeBron James"]

# Find the distance between lebron james and everyone else.
euclidean_distances = nba_normalized.apply(lambda row: distance.euclidean(row, lebron_normalized), axis=1)

# Create a new dataframe with distances.
distance_frame = pandas.DataFrame(data={"dist": euclidean_distances, "idx": euclidean_distances.index})
distance_frame.sort_values("dist", inplace=True)
# Find the most similar player to lebron (the lowest distance to lebron is lebron, the second smallest is the most similar non-lebron player)
second_smallest = distance_frame.iloc[1]["idx"]
most_similar_to_lebron = nba.loc[int(second_smallest)]["player"]
most_similar_to_lebron

'Carmelo Anthony'

In [91]:
import random
from numpy.random import permutation
#nba.drop('season')
# Randomly shuffle the index of nba.
random_indices = permutation(nba.index)
# Set a cutoff for how many items we want in the test set (in this case 1/3 of the items)
test_cutoff = math.floor(len(nba)/3)
# Generate the test set by taking the first 1/3 of the randomly shuffled indices.
test = nba.loc[random_indices[1:test_cutoff]]
# Generate the train set with the rest of the data.
train = nba.loc[random_indices[test_cutoff:]]
test = test.fillna(test.mean())
train = train.fillna(train.mean())

In [92]:
# The columns that we will be making predictions with.
x_columns = ['age', 'g', 'gs', 'mp', 'fg', 'fga', 'fg.', 'x3p', 'x3pa', 'x3p.', 'x2p', 'x2pa', 'x2p.', 'efg.', 'ft', 'fta', 'ft.', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf']
# The column that we want to predict.
y_column = ["pts"]

from sklearn.neighbors import KNeighborsRegressor
# Create the knn model.
# Look at the five closest neighbors.
knn = KNeighborsRegressor(n_neighbors=5)
# Fit the model on the training data.
knn.fit(train[x_columns], train[y_column])
# Make point predictions on the test set using the fit model.
predictions = knn.predict(test[x_columns])
print(predictions)

[[1.0120e+02]
 [1.3506e+03]
 [9.8000e+00]
 [6.6660e+02]
 [1.8400e+01]
 [1.4420e+02]
 [4.8000e+00]
 [1.3420e+02]
 [1.5040e+02]
 [9.5200e+01]
 [1.2900e+02]
 [1.0800e+02]
 [2.7400e+01]
 [1.2800e+01]
 [2.7600e+02]
 [8.7280e+02]
 [7.5640e+02]
 [7.8620e+02]
 [1.8800e+01]
 [9.0800e+01]
 [6.0480e+02]
 [1.1400e+01]
 [1.3440e+02]
 [8.8600e+01]
 [3.2420e+02]
 [1.8860e+02]
 [1.2260e+02]
 [1.9000e+01]
 [1.0772e+03]
 [3.4280e+02]
 [8.9840e+02]
 [1.6600e+01]
 [5.4620e+02]
 [6.9400e+01]
 [4.0200e+02]
 [4.2000e+00]
 [1.1800e+01]
 [2.0940e+02]
 [3.1820e+02]
 [1.5596e+03]
 [2.0020e+02]
 [9.9400e+01]
 [3.4640e+02]
 [5.7180e+02]
 [4.7660e+02]
 [1.8080e+02]
 [6.9300e+02]
 [3.4640e+02]
 [5.9960e+02]
 [3.5180e+02]
 [9.8440e+02]
 [9.1000e+01]
 [1.3616e+03]
 [3.1560e+02]
 [3.5100e+02]
 [1.2358e+03]
 [3.1880e+02]
 [9.2600e+01]
 [7.3700e+02]
 [5.3640e+02]
 [8.6000e+01]
 [1.0340e+03]
 [1.8000e+00]
 [2.9540e+02]
 [8.2060e+02]
 [4.1820e+02]
 [1.5208e+03]
 [1.5826e+03]
 [2.9400e+01]
 [9.6200e+01]
 [4.5320e+02]
 [6.65

In [93]:
    
# Get the actual values for the test set.
actual = test[y_column]

# Compute the mean squared error of our predictions.
mse = (((predictions - actual) ** 2).sum()) / len(predictions)
mse

pts    7537.543396
dtype: float64